In [5]:
from importlib import reload
import kc_functions as kc

/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
# df=pd.read_csv('data/2014/accident.csv')

In [8]:
# df.info()

In [4]:
# def getDistFromRow(dftmp,loc):
#     location=['LONGITUD','LATITUDE']
#     return np.linalg.norm(dftmp.loc[:,location]-loc,axis=1)
# 
# def LookupLocations(state,county,city,dflookup):
#     if dflookup is None:
#         dflookup=pd.read_csv(basefile,sep='|')
#     return dflookup.loc[(dflookup.STATE_NUMERIC==state) & (dflookup.COUNTY_NUMERIC==county),['PRIM_LONG_DEC','PRIM_LAT_DEC']].median()
# def fillLocations(dftmp,basefile='data/NationalFile_20190501.txt'):
#     dflup=pd.read_csv(basefile,sep='|')
#     state=dftmp.columns.tolist('STATE')
#     county=dftmp.columns.tolist('COUNTY')
#     city=dftmp.columns.tolist('CITY')
#     for i,v in dftmp.loc[(dftmp.LATITUDE>=99) & (dftmp.LONGITUD>=999),:].iterrows():
#         loc=LookupLocations(state,county,city,dflup)

### Binarize, cluster, and test for significance example - workflow

In [25]:
reload(kc)

df=kc.getalldata()
# need to add a merge to figure out how to merge sub-tables together into 1 row
df=kc.removeNoHourAndMinutes(df)
df=kc.createTimestamp(df)
df=kc.removeUnneededColumns(df)
df=kc.cleanLocs(df)

In [64]:
#kc.calculateTopNCatPct(df)
reload(kc)
kc.calculateTopNCatPct(df)

,variable,percent
0,state,0.394148
1,county,0.141392
2,city,0.539595
3,peds,0.999700
4,route,0.950530
5,harm_ev,0.760884
6,man_coll,0.994230
7,reljct2,0.977835
8,typ_int,0.998123
9,lgt_cond,0.996607


In [66]:
df.head().T

,0,1,2,3,4
latitude,34.6237,34.3974,33.1972,33.1964,34.1802
longitud,-85.9814,-87.7721,-87.5259,-86.3078,-86.7846
state,1,1,1,1,1
county,71,59,125,121,43
city,0,0,3050,2275,0
day,1,1,1,2,2
month,1,1,1,1,1
hour,1,13,3,9,16
peds,0,0,0,0,0
route,4,4,2,2,4


In [5]:
# df=kc_functions.binarizeVariables(df)
# df.info()

In [68]:
print(df.memory_usage(deep=True).sum())
#df.head().T

28961038


In [292]:
reload(kc)
hdbscan_params={'min_cluster_size':100,'gen_min_span_tree':False, 'metric':'manhattan'}
dfwk=kc.cluster_all_points(df,df.harm_ev==1,hdbscan_params=hdbscan_params)
dfwk=kc.ttest(dfwk,dfwk.harm_ev==1)
dfwk=kc.signficant_clusters(dfwk,True)

In [294]:
# reload(kc_functions)
# kc_functions.output_notebook()
#kc_functions.plot_map(dfwk)

In [295]:
# kc_functions.plot_map(dfwk,dfwk.cluster,dfwk.harm_ev==1)

In [296]:
dfwk=dfwk.loc[:,['id','cluster','label','tstat','pvalue','sig']]

In [300]:
# import tabpy as tabpy
reload(tabpy)

sigcluster,cluster,pvalue=tabpy.get_sig_clusters(df.longitud.values,df.latitude.values,(df.harm_ev==1).values,min_cluster_size=100,test_type='ttest')

{'min_cluster_size': 100, 'metric': 'manhattan', 'min_samples': 100}


In [301]:
all(dfwk['cluster']==sigcluster)

True

TypeError: '<' not supported between instances of 'list' and 'float'

In [26]:
import hdbscan

dfwk=df.loc[df.WRK_ZONE==1,LOCS]
clusterer = hdbscan.HDBSCAN(min_cluster_size=300, gen_min_span_tree=True,metric='manhattan')
clusterer.fit(dfwk)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=True, leaf_size=40,
    match_reference_implementation=False, memory=Memory(location=None),
    metric='manhattan', min_cluster_size=300, min_samples=None, p=None,
    prediction_data=False)

In [11]:
# plot_map(dfwk,clusterer.labels_,title='Fatal Accident Road condition Clusters')

In [214]:
# Workflow example
# z=cluster_all_points(df,df.WRK_ZONE==1)
# z=ttest(z,z.WRK_ZONE==1)
# z=signficant_clusters(z,2)

In [215]:
# z.cluster.value_counts()

## Workflow Example

In [14]:
df=getalldata()
#df=mergedata(df)
#df=cleanDFLocs(df)

In [ ]:
dfwk.cluster.value_counts()

In [ ]:
dfwk=create_plot_labels(dfwk)

In [ ]:
plot_map(dfwk,dfwk.plot_labels,title='Fatal Accident Road condition Clusters')

## End Example

In [122]:
#plot_map(dftmp,dftmp.cluster)

In [125]:
z=dfwk.loc[dfwk.cluster==3,LOCS]
hull=convexhull(z)
#hull=spatial.ConvexHull(z)
hull.close()

In [126]:
points=z.loc[:,LOCS].values

In [127]:
plt.plot(points[hull.vertices,0], points[hull.vertices,1], 'r--', lw=2)
plt.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro')
plt.plot(points[:,0],points[:,1],'bo')
plt.show()

In [128]:
z=df.loc[hull.in_hull(df.loc[:,LOCS].values),:]

In [129]:
from scipy.stats import ttest_ind

In [109]:
#ttest_ind(z.accident_WRK_ZONE,dfwk.loc[dfwk.cluster==-1,'accident_WRK_ZONE'])

In [16]:
# LOCS=['accident_LONGITUD', 'accident_LATITUDE','cluster']
# z.loc[:,LOCS].head()

In [139]:
df.loc[:,LOCS].head()

,accident_LONGITUD,accident_LATITUDE
0,-85.981408,34.623722
1,-87.772117,34.397428
2,-87.525911,33.197172
3,-86.307831,33.196383
4,-86.784592,34.180189


In [102]:
a=df.loc[~hull.in_hull(df.loc[:,LOCS].values),:]

In [18]:
# df.head().reset_index()

In [20]:
# df.loc[~hull.in_hull(df.loc[:,LOCS].values),'accident_WRK_ZONE'].mean()

In [94]:
df=getalldata()
# need to add a merge to figure out how to merge sub-tables together into 1 row
df=removeNoHourAndMinutes(df)
df=removeUneededAccidentColumns(df)
df=cleanLocs(df)
df=createTimestamp(df)

df2=getalldata()
# need to add a merge to figure out how to merge sub-tables together into 1 row
df2=removeNoHourAndMinutes(df2)
df2=removeUneededAccidentColumns(df2)
df2=cleanLocs(df2)
df2=createTimestamp(df2)
df=df.append(df2)
df=df.reset_index().drop('index',axis=1)

In [22]:
# print(df.shape)
# print(len(df.reset_index()['index'].unique()))
# print(len(df.reset_index().drop('index',axis=1).reset_index()['index'].unique()))
# df.sort_values('id').head()

In [6]:
reload(kc_functions)
df=kc_functions.getalldata()
df=kc_functions.removeNoHourAndMinutes(df)
df=kc_functions.createTimestamp(df)
df=kc_functions.removeUneededColumns(df)
df['target']=df['drunk_dr']

In [9]:
import pandas as pd
import kc_functions
df=pd.read_csv('comboacc.csv')
df.columns=[ i.lower() for i in df.columns.tolist()]
#df=kc_functions.removeNoHourAndMinutes(df)
#df=kc_functions.cleanLocs(df)
df['target']=df['sp_jur']
df.target.unique()

array([0, 9, 3, 5, 1, 8, 2, 4])

In [13]:
# ```SCRIPT_INT('from scipy.spatial import Delaunay, ConvexHull
from scipy.spatial import Delaunay,ConvexHull
from scipy.stats import ttest_ind
from scipy.spatial.qhull import QhullError
import pandas as pd
import hdbscan
import numpy as np

def in_hull(h,p):
    hull_in=Delaunay(h.points)
    return hull_in.find_simplex(p)>=0

LOCS=df.loc[:,["longitud","latitude","target"]]
# Assuming each row is unique
#LOCS = np.column_stack([np.radians(_arg1),np.radians(_arg2), _arg3]) 
uniquedf=pd.DataFrame(LOCS,columns=["longitud","latitude","target"])
# uniquedf is the uniques
targetdf = uniquedf.loc[uniquedf["target"]==1,:]
# clusterer = hdbscan.HDBSCAN(min_cluster_size=500,gen_min_span_tree=True,metric="manhattan",min_samples=500)
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,gen_min_span_tree=True,metric="manhattan",min_samples=5)
clustered = clusterer.fit(targetdf.loc[:,["longitud","latitude"]])
targetdf["cluster"]=clustered.labels_
uniquedf["cluster"]=0
# relabel cluster 0 so when we multiply it out to indicate significant clusters, it works - adding 1 to everything
uniquedf.loc[list(targetdf.index),"cluster"]=clustered.labels_ + 1
for i in uniquedf["cluster"].unique():
    if i!=0:
        dftgt=uniquedf.loc[uniquedf["cluster"]==i,["longitud","latitude"]]
        try:
            hull=ConvexHull(dftgt)
        except Exception as Err:
            hull=None
            if type(Err) == QhullError:
                if len(dftgt.longitud.unique())<3 and len(dftgt.latitude.unique())<3:
                    for lon,lat in zip(dftgt.longitud.unique(),dftgt.latitude.unique()):
                        uniquedf.loc[(uniquedf.longitud==lon) & (uniquedf.latitude==lat),"cluster"]==i
                else:
                    print(Err.args)   # True error here - we"ll have ot know about about it - not sure how you display it in tableau
        if hull is not None:
            hull.close()
            uniquedf.loc[in_hull(hull,uniquedf.loc[:,["longitud","latitude"]].values),"cluster"]=i
# ttest - no need for any correction because it"s different data sets
dfnotincluster=uniquedf.loc[uniquedf["cluster"]==0,"target"]
uniquedf["tstat"]=-1
uniquedf["pvalue"]=-1
for i in uniquedf["cluster"].unique():
    if i!=0:
        dfcluster=uniquedf.loc[uniquedf["cluster"]==i,"target"]
        tstat,pvalue=ttest_ind(dfcluster,dfnotincluster)
        uniquedf.loc[uniquedf["cluster"]==i,["tstat"]]=tstat
        uniquedf.loc[uniquedf["cluster"]==i,["pvalue"]]=pvalue
# now, check for significance
# uniquedf["sig"]=uniquedf.pvalue<= 0.05 
uniquedf["sig"]=uniquedf.pvalue<=.05
uniquedf["sig_cluster"]=uniquedf.cluster*uniquedf.sig


# Showing certain clusters show up as significant that have no difference (likely due to chance). 
# ',
# avg([Longitud]), 
# avg([Latitude]),
# avg([Measure Chooser]),
# min([# of clusters]))```

/home/kchuangk/.pyenv/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


QhullError: QH6013 qhull input error: input is less than 2-dimensional since it has the same x coordinate

While executing:  | qhull i Qt
Options selected for Qhull 2015.2.r 2016/01/18:
  run-id 1991766488  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  0  Error-roundoff 7.2e-13  _one-merge 3.6e-12
  _near-inside 1.8e-11  Visible-distance 1.4e-12  U-coplanar-distance 1.4e-12
  Width-outside 2.9e-12  _wide-facet 8.6e-12


In [15]:
# Final for Tabpy

from scipy.spatial import Delaunay, ConvexHull
from scipy.stats import ttest_ind
from scipy.spatial.qhull import QhullError
import pandas as pd
import hdbscan

def in_hull(h,p):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    hull_in=Delaunay(h.points)
    return hull_in.find_simplex(p)>=0

LOCS=df.loc[:,['longitud','latitude','target']]
# Assuming each row is unique
# TABPY LOCS = np.column_stack([np.radians(_arg1),np.radians(_arg2), _arg3]) 
uniquedf=pd.DataFrame(LOCS,columns=['longitud','latitude','target'])
# uniquedf is the uniques
targetdf = uniquedf.loc[uniquedf['target']==1,:]
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,gen_min_span_tree=True,metric="manhattan",min_samples=5)
#clusterer = hdbscan.HDBSCAN(min_cluster_size=_arg4[1],gen_min_span_tree=True,metric="manhattan",min_samples=_arg5[1])
clustered = clusterer.fit(targetdf.loc[:,['longitud','latitude']])
targetdf['cluster']=clustered.labels_
uniquedf['cluster']=0
# relabel cluster 0 so when we multiply it out to indicate significant clusters, it works - adding 1 to everything
uniquedf.loc[list(targetdf.index),'cluster']=clustered.labels_ + 1
for i in uniquedf['cluster'].unique():
    if i!=0:
        dftgt=uniquedf.loc[uniquedf['cluster']==i,['longitud','latitude']]
        try:
            hull=ConvexHull(dftgt)
        except Exception as Err:
            hull=None
            if type(Err) == QhullError:
                if len(dftgt.longitud.unique())<3 and len(dftgt.latitude.unique())<3:
                    for lon,lat in zip(dftgt.longitud.unique(),dftgt.latitude.unique()):
                        uniquedf.loc[(uniquedf.longitud==lon) & (uniquedf.latitude==lat),'cluster']==i
                else:
                    print(Err.args)   # True error here - we'll have ot know about about it - not sure how you display it in tableau
        if hull is not None:
            hull.close()
            uniquedf.loc[in_hull(hull,uniquedf.loc[:,['longitud','latitude']].values),'cluster']=i
# ttest - no need for any correction because it's different data sets
dfnotincluster=uniquedf.loc[uniquedf['cluster']==0,'target']
uniquedf['tstat']=-1
uniquedf['pvalue']=-1
for i in uniquedf['cluster'].unique():
    if i!=0:
        dfcluster=uniquedf.loc[uniquedf['cluster']==i,'target']
        tstat,pvalue=ttest_ind(dfcluster,dfnotincluster)
        uniquedf.loc[uniquedf['cluster']==i,['tstat']]=tstat
        uniquedf.loc[uniquedf['cluster']==i,['pvalue']]=pvalue
# now, check for significance
uniquedf['sig']=uniquedf.pvalue<= 0.03 
# TABPY uniquedf['sig']=uniquedf.pvalue<=_arg6[1]
uniquedf['sig_cluster']=uniquedf.cluster*uniquedf.sig

# PLEASE READ: basically cluster 0 means it's not a cluster - everythign else, it's in a cluster
# fulldf has every single point (repeated) - cluster is the cluster it is in 0 means not in a cluster or significant cluster, -2 means repeated
# uniquedf has only the unique points - sig_cluster is the significant cluster 0 means it's not in a cluster, cluster = cluster from hdbscan (no significance)
# return uniquedf['sig']

/home/kchuangk/.pyenv/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
# kc_functions.output_notebook()
# kc_functions.plot_map(uniquedf,uniquedf.cluster)

In [26]:
# kc_functions.plot_map(uniquedf,uniquedf.sig_cluster) #,uniquedf.target)

### Data Merge

In [111]:
varlist=getVarlist()

In [11]:
varlist

In [28]:
# reload(kc_functions)
# a=kc_functions.createHistograms('accident.csv')

In [201]:
# we should probably just bin the Vehicle Number

In [202]:
df=getalldata('person.csv')

In [218]:
z=df.loc[:,varlist['person']].groupby('id')['per_no'].count()
(z.value_counts()/z.shape[0]).values.cumsum()

array([0.38010776, 0.65283572, 0.82420263, 0.90564561, 0.94857473,
       0.97152498, 0.98336487, 0.99001436, 0.9934417 , 0.99558474,
       0.99693743, 0.99779617, 0.99825973, 0.99858651, 0.99881449,
       0.99901208, 0.99920966, 0.99932365, 0.99939965, 0.99946804,
       0.99952124, 0.99957443, 0.99960483, 0.99963523, 0.99966563,
       0.99969602, 0.99971882, 0.99974162, 0.99976442, 0.99977962,
       0.99979482, 0.99981001, 0.99982521, 0.99984041, 0.99985561,
       0.99987081, 0.99988601, 0.99990121, 0.99991641, 0.99992401,
       0.99993161, 0.9999392 , 0.9999468 , 0.9999544 , 0.999962  ,
       0.9999696 , 0.9999772 , 0.9999848 , 0.9999924 , 1.        ])

## EDA

In [30]:
# kc_functions.output_notebook()
# kc_functions.createHistograms('accident.csv')

In [32]:
# kc_functions.createHistograms('damage.csv')

In [34]:
# kc_functions.createHistograms('vision.csv')

In [36]:
# kc_functions.createHistograms('drimpair.csv')

In [38]:
# kc_functions.createHistograms('factor.csv')

In [40]:
# kc_functions.createHistograms('person.csv')

In [42]:
# kc_functions.createHistograms('acc_aux.csv')

In [44]:
# from bokeh.models.annotations import Span
# 
# k=[i/100 for i in range(0,int(2*np.pi*100),10)]
# z=np.sin(k)
# 
# p=kc_functions.figure(tooltips=[('x','$x'),('y','$y')])
# p.line(x=k,y=z,legend='sin')
# p.circle(x=k,y=z,legend='sin',radius=0.10,fill_color=None)
# s=Span(location=0,dimension='width',line_width=1,line_color='darkred',line_dash='dotted')
# p.add_layout(s)
# s=Span(location=3,dimension='height',line_width=1,line_color='navy',line_dash='dashed')
# p.add_layout(s)
# kc_functions.show(p)

### Create Functions to merge data for EDA in R

In [2]:
# load values for R
import kc_functions
from importlib import reload
reload(kc_functions)
accident=kc_functions.getalldata('accident.csv')
accident=kc_functions.removeNoHourAndMinutes(accident)
accident=kc_functions.cleanLocs(accident)
accident=kc_functions.createTimestamp(accident)
accident=kc_functions.removeUnneededColumns(accident,'accident.csv',['tstamp'])
vehicle=kc_functions.getalldata('vehicle.csv')
vehicle=kc_functions.removeUnneededColumns(vehicle,'vehicle.csv')
veh_aux=kc_functions.getalldata('veh_aux.csv')
veh_aux=kc_functions.removeUnneededColumns(veh_aux,'veh_aux.csv')

/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (36,37,102) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
accident=accident.set_index('id')
vehicle.loc[:,'tstamp']=accident.loc[list(vehicle.id),'tstamp'].values
vehicle.loc[:,'state']=accident.loc[list(vehicle.id),'state'].values
vehicle.loc[:,'longitud']=accident.loc[list(vehicle.id),'longitud'].values
vehicle.loc[:,'latitude']=accident.loc[list(vehicle.id),'latitude'].values
veh_aux.loc[:,'state']=accident.loc[list(veh_aux.id),'state'].values
veh_aux.loc[:,'tstamp']=accident.loc[list(veh_aux.id),'tstamp'].values
veh_aux.loc[:,'longitud']=accident.loc[list(veh_aux.id),'longitud'].values
veh_aux.loc[:,'latitude']=accident.loc[list(veh_aux.id),'latitude'].values

/home/kchuangk/.pyenv/lib/python3.6/site-packages/pandas/core/indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [4]:
vehicle=vehicle.loc[(~vehicle.tstamp.isnull()) | (~vehicle.longitud.isnull()) | (~vehicle.latitude.isnull()),:]
veh_aux=veh_aux.loc[(~veh_aux.tstamp.isnull()) | (~veh_aux.longitud.isnull()) | (~veh_aux.latitude.isnull()),:]

In [5]:
vehicle.to_csv('kc_eda_vehicle.csv')
veh_aux.to_csv('kc_eda_veh_aux.csv')

The rest of the EDA is done in R

### EDA Tranformations below

We'll code only examining vehicles 1-4 (~94% of all accidents) and vehicles 1-4 covers 98-99% of variables

We'll code each categorical variable as veh_no_(1-4)_categorical_var_1,veh_no_(1-4)_categorical_var_2, and so on and numerical as veh_no_(1-4)_numericVar.

Note veh_aux doesn't have vehicle number.

In [8]:
from importlib import reload
import pandas as pd
import kc_functions as kc

/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
# dforig=pd.read_csv('kc_eda_vehicle.csv')
# df=dforig.copy()
# cols=[i for i in df.columns.tolist() if i not in ['tstamp','longitud','latitude','Unnamed: 0']]
# df=df.loc[:,cols]
# df=df.loc[(df.veh_no<=4),:]
# df['veh_no']=['veh'+str(i) for i in df.veh_no]
# df=df.drop('state',axis=1)

In [10]:
from importlib import reload
import numpy as np
import kc_functions as kc
reload(kc)

<module 'kc_functions' from '/home/kchuangk/Documents/Classes/W210 - Capstone/NYCTrafficCollisions/kc/kc_functions.py'>

In [15]:
reload(kc)
#kc.createDataDictForTranslation()
vehicle=kc.getalldata('vehicle.csv')
vehicle=kc.removeUnneededColumns(vehicle,'vehicle')
vehicle=vehicle.drop(['unittype','spec_use','underride', 'vpavetype'],axis=1)   # columns dropped from lack of variance in additional EDA
# map the variables into bins/translations
vehicle=vehicle.loc[(vehicle.veh_no<=4),:]
vehicle=kc.translateVarsFromDict(vehicle,kc.createDataDictForTranslation('vehicle'))
vehicle['veh_no']=['veh'+str(int(i)) for i in vehicle.veh_no]
vehicle=kc.pivot_and_chunk(vehicle)

In [16]:
vehicle.columns

Index(['numoccs$veh1', 'numoccs$veh2', 'numoccs$veh3', 'numoccs$veh4',
       'unittype$veh1', 'unittype$veh2', 'unittype$veh3', 'unittype$veh4',
       'hit_run$veh1', 'hit_run$veh2',
       ...
       'pcrash4$veh3', 'pcrash4$veh4', 'pcrash5$veh1', 'pcrash5$veh2',
       'pcrash5$veh3', 'pcrash5$veh4', 'acc_type$veh1', 'acc_type$veh2',
       'acc_type$veh3', 'acc_type$veh4'],
      dtype='object', length=128)

In [17]:
# remove variables not to expand
cols=vehicle.columns.tolist()
for k,v in kc.createDataDictForTranslation().items():   # filtering out only columns to be binarized
    if k in kc.checkDataDictForType(SourceFile='vehicle',return_type='numeric'):
        for i in range(1,5):
            cols.remove(k+'$veh'+str(i))

In [18]:
# binarize the variables
vehicle=kc.binarizeVariables(vehicle,cols,500)
# vehicles is ready for merging
vehicle.head()

,numoccs$veh1,numoccs$veh2,numoccs$veh3,numoccs$veh4,unittype$veh1,unittype$veh2,unittype$veh3,unittype$veh4,hit_run$veh1,hit_run$veh2,...,acc_type$veh3_1.0,acc_type$veh3_13.0,acc_type$veh3_99.0,acc_type$veh4_0.0,acc_type$veh4_98.0,acc_type$veh4_50.0,acc_type$veh4_1.0,acc_type$veh4_6.0,acc_type$veh4_13.0,acc_type$veh4_51.0
id,,,,,,,,,,,,,,,,,,,,,
id_2014.100001,99.0,99.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
id_2014.100002,99.0,99.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
id_2014.100003,99.0,99.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
id_2014.100004,99.0,99.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
id_2014.100005,99.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,True,False,False,False,False,False,False


# **PLEASE LOOK - Use the workflow below to process to file as a template**

In [577]:
from importlib import reload
import kc_functions as kc
reload(kc)

<module 'kc_functions' from '/home/kchuangk/Documents/Classes/W210 - Capstone/NYCTrafficCollisions/kc/kc_functions.py'>

In [579]:
from importlib import reload
import kc_functions as kc

reload(kc)
#kc.createDataDictForTranslation()
vehicle=kc.getalldata('vehicle.csv')
vehicle=kc.removeUnneededColumns(vehicle,'vehicle')
vehicle=vehicle.drop(['unittype','spec_use','underide','vpavetyp'],axis=1)   # columns dropped from lack of variance in additional EDA
# map the variables into bins/translations
vehicle=vehicle.loc[(vehicle.veh_no<=4),:]
vehicle=kc.translateVarsFromDict(vehicle,kc.createDataDictForTranslation('vehicle'))
vehicle['veh_no']=['veh'+str(int(i)) for i in vehicle.veh_no]
vehicle=kc.pivot_and_chunk(vehicle)
# remove variables not to expand
cols=vehicle.columns.tolist()
for k,v in kc.createDataDictForTranslation().items():   # filtering out only columns to be binarized
    if k in kc.checkDataDictForType(SourceFile='vehicle',return_type='numeric'):
        for i in range(1,5):
            cols.remove(k+'$veh'+str(i))
vehicle=kc.binarizeVariables(vehicle,cols,500)    # binarize the columns
vehicle.apply(kc.addSummaryStats,axis=1,functions=[np.nanmean,np.nansum,np.any],postfixes=['mean','sum','any'])

/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (36,37,102) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


KeyboardInterrupt: 

In [ ]:
#vehicle.head().T

In [ ]:
# procedure to load the file all the way to prepped

reload(kc)
veh_aux=kc.getalldata('veh_aux.csv')  # load all year's data and add id
veh_aux=kc.removeUnneededColumns(veh_aux,'veh_aux',['veh_no'])  # keep veh_no
#veh_aux=kc.drop([],axis=1) # drop additional columns from deeper EDA for lack of variance ien the variables
veh_aux=veh_aux.loc[(veh_aux.veh_no<=4),:]    # filter only veh_no <=4
veh_aux=kc.translateVarsFromDict(veh_aux,kc.createDataDictForTranslation('veh_aux'))   #remap the values from FARS to ours own values
veh_aux['veh_no']=['veh'+str(int(i)) for i in veh_aux.veh_no]    # make veh_no a string instead
veh_aux=kc.pivot_and_chunk(veh_aux)    # pivot the veh_no from values into columns headers (expanding number of columns by columns * veh_no.unique()
cols=veh_aux.columns.tolist()          # Filtering out columns to binarized -- only categorical columns
for k,v in kc.createDataDictForTranslation().items():
    if k in kc.checkDataDictForType(SourceFile='veh_aux',return_type='numeric'):
        for i in range(1,5):
            cols.remove(k+'$veh'+str(i))
veh_aux=kc.binarizeVariables(veh_aux,cols,500)    # binarize the columns
veh_aux.apply(kc.addSummaryStats,axis=1,functions=[np.nanmean,np.nansum,np.any],postfixes=['mean','sum','any'])

In [ ]:
#veh_aux.head()

In [ ]:
print(len(vehicle.reset_index().id.unique())==vehicle.shape[0])
print(vehicle.shape[0])

In [ ]:
print(len(veh_aux.reset_index().id.unique())==veh_aux.shape[0])
print(veh_aux.shape[0])

#### Merging with accident file

In [ ]:
accident=kc.getalldata()
accident=kc.removeNoHourAndMinutes(accident)
accident=kc.cleanLocs(accident)

In [ ]:
accident=kc.removeUnneededColumns(accident)

In [ ]:
accident.shape

In [ ]:
accident=accident.set_index('id')
accident=accident.join(vehicle,how='left')

In [ ]:
accident=accident.join(veh_aux,how='left')

In [ ]:
accident.shape

In [ ]:
# check for duplicates
any(accident.reset_index().id.duplicated())

In [ ]:
accident.to_csv('kc_vehicle_accident_veh_aux.csv')

In [3]:
from importlib import reload
import kc_functional_functions as kc

/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [33]:
from importlib import reload
import kc_functions as kc

In [35]:
df=pd.read_csv('kc_vehicle_accident_veh_aux.csv')

/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,2

In [37]:
df.head().T

,0,1,2,3,4
id,id_2014.10001,id_2014.10002,id_2014.10003,id_2014.10004,id_2014.10005
latitude,34.6237,34.3974,33.1972,33.1964,34.1802
longitud,-85.9814,-87.7721,-87.5259,-86.3078,-86.7846
state,1,1,1,1,1
county,71,59,125,121,43
city,0,0,3050,2275,0
day,1,1,1,2,2
month,1,1,1,1,1
hour,1,13,3,9,16
peds,0,0,0,0,0


In [5]:
import numpy as np
import pandas as pd

In [7]:
import kc_functions as kc

/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/kchuangk/.pyenv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [13]:
%config IPCompleter.greedy=True
%config IPCompleter.greedy=True

np.

In [19]:
!pip3 install numpy

  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
basemap 1.2.0 requires matplotlib>=1.0.0, which is not installed.
basemap 1.2.0 requires pyproj>=1.9.3, which is not installed.
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/lib/python3.6/site-packages/numpy-1.16.4.dist-info'
Consider using the `--user` option or check the permissions.

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
accident.head().T

id,id_2014.10001,id_2014.10002,id_2014.10003,id_2014.10004,id_2014.10005
latitude,34.6237,34.3974,33.1972,33.1964,34.1802
longitud,-85.9814,-87.7721,-87.5259,-86.3078,-86.7846
state,1,1,1,1,1
county,71,59,125,121,43
city,0,0,3050,2275,0
day,1,1,1,2,2
month,1,1,1,1,1
hour,1,13,3,9,16
peds,0,0,0,0,0
route,4,4,2,2,4


In [49]:
import re

In [53]:
accident.columns.tolist()

['latitude',
 'longitud',
 'state',
 'county',
 'city',
 'day',
 'month',
 'hour',
 'peds',
 'route',
 'harm_ev',
 'man_coll',
 'reljct2',
 'typ_int',
 'lgt_cond',
 'weather',
 'cf1',
 'drunk_dr',
 'numoccs$veh1',
 'numoccs$veh2',
 'numoccs$veh3',
 'numoccs$veh4',
 'reg_stat$veh1',
 'reg_stat$veh2',
 'reg_stat$veh3',
 'reg_stat$veh4',
 'make$veh1',
 'make$veh2',
 'make$veh3',
 'make$veh4',
 'mod_year$veh1',
 'mod_year$veh2',
 'mod_year$veh3',
 'mod_year$veh4',
 'trav_sp$veh1',
 'trav_sp$veh2',
 'trav_sp$veh3',
 'trav_sp$veh4',
 'vnum_lan$veh1',
 'vnum_lan$veh2',
 'vnum_lan$veh3',
 'vnum_lan$veh4',
 'vspd_lim$veh1',
 'vspd_lim$veh2',
 'vspd_lim$veh3',
 'vspd_lim$veh4',
 'hit_run$veh1_No',
 'hit_run$veh1_Yes',
 'hit_run$veh1_Unknown',
 'hit_run$veh2_No',
 'hit_run$veh2_Yes',
 'hit_run$veh2_Unknown',
 'hit_run$veh3_No',
 'hit_run$veh3_Yes',
 'hit_run$veh3_Unknown',
 'hit_run$veh4_No',
 'hit_run$veh4_Yes',
 'hit_run$veh4_Unknown',
 'owner$veh1_Private Owner',
 'owner$veh1_Business/Governme

In [59]:
accident.loc[:,[True if re.search('a_drdro',i) else False for i in accident.columns.tolist()]].T

id,id_2014.10001,id_2014.10002,id_2014.10003,id_2014.10004,id_2014.10005,id_2014.10006,id_2014.10007,id_2014.10008,id_2014.10009,id_2014.10010,...,id_2017.560096,id_2017.560097,id_2017.560098,id_2017.560099,id_2017.560100,id_2017.560101,id_2017.560102,id_2017.560103,id_2017.560104,id_2017.560105
a_drdro$veh1_No,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
a_drdro$veh1_Drowsy Driver,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$veh2_No,False,False,True,True,False,False,True,True,True,True,...,False,False,False,True,False,False,False,False,False,False
a_drdro$veh2_Drowsy Driver,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$veh3_No,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$veh3_Drowsy Driver,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$veh4_No,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$veh4_Drowsy Driver,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
a_drdro$mean,0.125,0.125,0.25,0.375,0.125,0.125,0.25,0.25,0.375,0.25,...,0.125,0.125,0.125,0.25,0.125,0.125,0.125,0.125,0.125,0.125
a_drdro$sum,1,1,2,3,1,1,2,2,3,2,...,1,1,1,2,1,1,1,1,1,1


In [61]:
cols=accident.columns.tolist()

In [63]:
cols

['latitude',
 'longitud',
 'state',
 'county',
 'city',
 'day',
 'month',
 'hour',
 'peds',
 'route',
 'harm_ev',
 'man_coll',
 'reljct2',
 'typ_int',
 'lgt_cond',
 'weather',
 'cf1',
 'drunk_dr',
 'numoccs$veh1',
 'numoccs$veh2',
 'numoccs$veh3',
 'numoccs$veh4',
 'reg_stat$veh1',
 'reg_stat$veh2',
 'reg_stat$veh3',
 'reg_stat$veh4',
 'make$veh1',
 'make$veh2',
 'make$veh3',
 'make$veh4',
 'mod_year$veh1',
 'mod_year$veh2',
 'mod_year$veh3',
 'mod_year$veh4',
 'trav_sp$veh1',
 'trav_sp$veh2',
 'trav_sp$veh3',
 'trav_sp$veh4',
 'vnum_lan$veh1',
 'vnum_lan$veh2',
 'vnum_lan$veh3',
 'vnum_lan$veh4',
 'vspd_lim$veh1',
 'vspd_lim$veh2',
 'vspd_lim$veh3',
 'vspd_lim$veh4',
 'hit_run$veh1_No',
 'hit_run$veh1_Yes',
 'hit_run$veh1_Unknown',
 'hit_run$veh2_No',
 'hit_run$veh2_Yes',
 'hit_run$veh2_Unknown',
 'hit_run$veh3_No',
 'hit_run$veh3_Yes',
 'hit_run$veh3_Unknown',
 'hit_run$veh4_No',
 'hit_run$veh4_Yes',
 'hit_run$veh4_Unknown',
 'owner$veh1_Private Owner',
 'owner$veh1_Business/Governme

In [233]:
re.search('(.*)\$veh[1-4]_{0,1}(.*)','acc_type$veh3_Goes a bit crazy, top 5').groups()

('acc_type', 'Goes a bit crazy, top 5')

In [78]:
z=pd.DataFrame([[1,2,3],[4,5,6]],columns=['a','b','c'])

In [80]:
z

,a,b,c
0,1,2,3
1,4,5,6


In [104]:
z.apply(lambda x: x.index.tolist(),axis=1)

0    [a, b, c]
1    [a, b, c]
dtype: object

In [118]:
re.search('(.*)\$veh[1-4]$','reg_stat$veh2').groups()

('reg_stat',)

In [278]:
reload(kc)
a=accident.head(1).apply(kc.addSummaryStats,axis=1,functions=[np.sum],postfixes=['sum'])

In [433]:
np.any

<function numpy.any(a, axis=None, out=None, keepdims=<no value>)>

In [537]:
r=accident.iloc[0,:]

In [571]:
reload(kc)
a=kc.addSummaryStats(r,[np.nansum,np.nanmean,np.any],['sum','mean','any'])

processing rolinloc$Unkown$sum
processing rolinloc$Unkown$mean
processing rolinloc$Unkown$any
processing p_crash1$Turning Left$sum
processing p_crash1$Turning Left$mean
processing p_crash1$Turning Left$any
processing p_crash2$Pedalcyclist or Other Non-Motorist in Road$sum
processing p_crash2$Pedalcyclist or Other Non-Motorist in Road$mean
processing p_crash2$Pedalcyclist or Other Non-Motorist in Road$any
processing p_crash2$From Parking Lane/Shoulder, Median/Crossover, Roadside$sum
processing p_crash2$From Parking Lane/Shoulder, Median/Crossover, Roadside$mean
processing p_crash2$From Parking Lane/Shoulder, Median/Crossover, Roadside$any
processing deformed$Disabling Damage$sum
processing deformed$Disabling Damage$mean
processing deformed$Disabling Damage$any
processing p_crash1$No Driver Present/Unknown if Driver Present$sum
processing p_crash1$No Driver Present/Unknown if Driver Present$mean
processing p_crash1$No Driver Present/Unknown if Driver Present$any
processing p_crash2$From 

In [575]:
a[[True if re.search('numoccs',i) else False for i in a.index.tolist()]]

numoccs$veh1        2
numoccs$veh2      NaN
numoccs$veh3      NaN
numoccs$veh4      NaN
numoccs$mean        2
numoccs$sum         2
numoccs$any      True
numoccs$$sum        2
numoccs$$mean       2
numoccs$$any     True
Name: id_2014.10001, dtype: object

In [468]:
np.nansum(a.astype('int'))

0

In [466]:
np.nanmean(a.astype('int'))

0.0

In [488]:
np.nanmean(np.array([np.nan,10,10]).astype('int'))

-3.0744573456182584e+18

In [510]:
a=pd.Series([np.nan,0,1,True,10,False])

In [514]:
a.index=['a','b','c','d','e','f']

In [516]:
a

a      NaN
b        0
c        1
d     True
e       10
f    False
dtype: object

In [528]:
np.nanmean(a.dropna().astype('int'))

2.4

In [526]:
a.dropna()

b        0
c        1
d     True
e       10
f    False
dtype: object

2.4